In [1]:
import _ct_district_grain_data_prep as _ct_district_data_prep
import geopandas as gpd
import pandas as pd
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
from shared_utils import webmap_utils
from calitp_data_analysis import geography_utils
import branca
import branca.colormap as cm

In [2]:
import google.auth
credentials, project = google.auth.default()
import gcsfs
fs = gcsfs.GCSFileSystem()

In [3]:
district = "04 - Bay Area / Oakland"

In [4]:
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [5]:
boundary_file = f"district_{district_int}_boundary"
transit_routes_file = f"district_{district_int}_transit_routes"
shn_file = f"district_{district_int}_shn"
transit_shn_file = f"district_{district_int}_transit_routes_shn"

In [6]:
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)

In [7]:
district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [8]:
operator_gdf = _ct_district_data_prep.data_wrangling_operator_map(
    list(operator_df.analysis_name.unique())
)

In [9]:
color_map = cm.linear.Spectral_11.scale(0, 50)

In [10]:
color_map

In [11]:
# operator_gdf[["number"]]

In [12]:
# color_map = cm.linear.Spectral_11.scale(0, 50)  # adjust the range as needed
color_map = color_map.to_step(25)  # adjust the number of steps as needed

In [13]:
color_map

In [14]:
district_map = webmap_utils.set_state_export(
    district_gdf,
    subfolder = "caltrans_district_digest/",
    filename=boundary_file,
    map_title="District Map",
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [15]:
transit_routes = webmap_utils.set_state_export(
    operator_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_routes_file,
    map_title="Transit Routes Title 2",
    cmap=color_map,
    color_col="number",
    existing_state=district_map
    
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [16]:
webmap_utils.render_spa_link(transit_routes["spa_link"], text="Open Routes for all Operators Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJEaXN0cmljdCBNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2NhbHRyYW5zX2Rpc3RyaWN0X2RpZ2VzdC9kaXN0cmljdF80X2JvdW5kYXJ5Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19LCB7Im5hbWUiOiAiVHJhbnNpdCBSb3V0ZXMgVGl0bGUgMiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvY2FsdHJhbnNfZGlzdHJpY3RfZGlnZXN0L2Rpc3RyaWN0XzRfdHJhbnNpdF9yb3V0ZXMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNy44NzQ4MjQ1MzQxNzc3MjYsIC0xMjIuMjQyNTE4MDcxNTc2MzRdLCAiem9vbSI6IDEzfQ==" target="_blank">Open Open Routes for all Operators Map in New Tab</a>

In [17]:
webmap_utils.display_spa_map(transit_routes["spa_link"])